# TensorFlow Assignment: Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Khoi D. Vo </br>
Date: June 25, 2018

### Convolutional Neural Network

Build a 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> Convolution (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> Convolution (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (256 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some tips:
- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- Since CNNs a more complex than the logistic regression and MLP models you've worked with before, so you may find it helpful to use a more advanced optimizer. You're model will train faster if you use [`tf.train.AdamOptimizer`](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) instead of `tf.train.GradientDescentOptimizer`. A learning rate of 1e-4 is a good starting point.

#### Importing necessary toolboxes & image sets

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
from tqdm import trange         

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("official/mnist/dataset.py", one_hot=True) 

Extracting official/mnist/dataset.py/train-images-idx3-ubyte.gz
Extracting official/mnist/dataset.py/train-labels-idx1-ubyte.gz
Extracting official/mnist/dataset.py/t10k-images-idx3-ubyte.gz
Extracting official/mnist/dataset.py/t10k-labels-idx1-ubyte.gz


#### Constructing the model

In [16]:
tf.reset_default_graph()

def conv(x, w, b, s):
    x = tf.nn.conv2d(x, w, strides=[1,s,s,1], padding='SAME')
    x = tf.nn.relu(tf.add(x, b))
    return x

def conv_network(x, w, b, s):
    x = tf.reshape(x, [-1, 28, 28, 1])
    
    # convolution layer 1
    conv1 = conv(x, w['wc_1'], b['bc_1'], s) 
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 
    
    # convolution layer 2
    conv2 = conv(conv1, w['wc_2'], b['bc_2'], s) # [-1,14,14,64]
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 

    # fully connected layers
    fc_1 = tf.reshape(conv2, [-1, w['wfc_1'].get_shape().as_list()[0]])
    fc_1 = tf.nn.relu(tf.matmul(fc_1, w['wfc_1']) + b['bfc_1'])
    fc_out = tf.matmul(fc_1, w['wfc_out']) + b['bfc_out']
    
    return fc_out

# Defining weights and biases
weights = {'wc_1': tf.Variable(tf.random_normal([5, 5, 1, 32])), # convolution filter 1
           'wc_2': tf.Variable(tf.random_normal([5, 5, 32, 64])), # convolution filter 2
           'wfc_1': tf.Variable(tf.random_normal([7*7*64, 256])), # fully connected, 7*7*64 inputs (two poolings of 2x2, original image 28x28 --> 7x7x64), 256 outputs
           'wfc_out': tf.Variable(tf.random_normal([256, 10]))}

biases = {'bc_1': tf.Variable(tf.random_normal([32])),
          'bc_2': tf.Variable(tf.random_normal([64])),
          'bfc_1': tf.Variable(tf.random_normal([256])),
          'bfc_out': tf.Variable(tf.random_normal([10]))}

stride = 1

# Define input placeholder & logit
X = tf.placeholder(tf.float32, [None, 784]) # 784 pixels per image
logits = conv_network(X, weights, biases, stride)

# Define loss and optimizer functions
Y = tf.placeholder(tf.float32, [None, 10])
loss_fn = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer_fn = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_fn)

#### Training the model

In [17]:
# Create a session object and initialize all graph variables
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [18]:
# Train the model
batch_size = 1000
for epoch in trange(500):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(optimizer_fn, feed_dict={X: batch_x, Y: batch_y})

100%|██████████| 500/500 [09:04<00:00,  1.09s/it]


#### Testing the model

In [19]:
# Test trained model
prediction = tf.nn.softmax(logits)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Test accuracy: {0}' \
      .format(sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})))

Test accuracy: 0.9697999954223633
